<a href="https://colab.research.google.com/github/kmouts/PPS_MultiComms/blob/master/video_analytics_with_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ανάλυση βίντεο με τεχνικές Βαθιάς Μάθησης

## Εισαγωγή
Σε αυτό το εργαστήριο θα χρησιμοποιήσουμε τεχνικές Deep Learning για να αναγνωρίσουμε διαφορετικούς χαρακτήρες σε ένα βίντεο, και το χρόνο που παρουσιάζεται κάθε ένας. Θα δουλέψουμε με Python σε ένα απλό βίντεο κινούμενων σχεδίων! (Tom & Jerry).

Τα βήματα που θα ακολουθήσουμε είναι τα εξής:

1.   Διάβασμα βίντεο και εξαγωγή  σκηνών (frames)
2.   Εξαγωγή χαρακτηριστικών - Εκπαίδευση μοντέλου
3.   Υπολογισμός σκηνικού χρόνου - Μια απλή λύση
4.   Σκέψεις για βελτίωση

## 1.   Διάβασμα βίντεο και εξαγωγή  σκηνών (frames)
Το βίντεο δεν είναι τίποτε παραπάνω από μια συλλογή εικόνων. Αυτές οι εικόνες ονομάζονται σκηνές (frames) και μπορούν να συνδιαστούν για να πάρουμε το αρχικό βίντεο. Ετσι, το πρόβλημα σε σχέση με δεδομένα βίντεο δεν είναι τόσο διαφορετικό απο την ταξινόμηση εικόνας ή την αναγνώριση αντικειμένων. Απλώς υπάρχει ένα επιπλέον βήμα του τα εξάγουμε σκηνές από το βίντεο.

Μιάς και το πρόβλημά μας είναι να υπολογίσουμε το χρόνο κάθε χαρακτήρα στο βίντεο, ας δούμε πιο αναλυτικά τα βήματα που θα ακολουθήσουμε:


*   Εισαγωγή και ανάγνωση του βίντεο, εξαγωγή σκηνών και αποθήκευση ως εικόνες.
*   Χαρακτηρισμός κάποιων εικόνων για εκπαίδευση του μοντέλου.
*   Δημιουργία μοντέλου με εκπαίδευση.
*   Προβλέψεις για νέες εικόνες.
*   Υπολογισμός σκηνικού χρόνου.





##Χειρισμός αρχείων βίντεο με Python
Ξεκινάμε με την εισαγωγή απαραίτητων βιβλιοθηκών:

    Numpy
    Pandas
    Matplotlib
    Keras
    Skimage
    OpenCV


In [ ]:
!apt install ffmpeg
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images



### Βήμα – 1: Διάβασμα βίντεο, εξαγωγή σκηνών και αποθήκευση ως εικόνες

Ας κατεβάσουμε το βίντεο με την συνάρτηση `wget`.

In [ ]:
#!pip install gdown
import gdown

url = 'https://github.com/kmouts/PPS_MultiComms/blob/master/Tom_jerry.mp4?raw=true'
output = 'Tom_jerry.mp4'
gdown.download(url, output, quiet=False)

In [ ]:
!ls -lah

Θα φορτώσουμε το βίντεο με τη συνάρτηση VideoCapture() και θα το μετατρέψουμε σε σκηνές (εικόνες) που θα τις σώσουμε με την συνάρτηση imwrite().

In [ ]:
import os
if not os.path.isdir( 'tom' ) :
    os.mkdir( 'tom' )  # make sure the directory exists

In [ ]:
count = 0
videoFile = "Tom_jerry.mp4"
cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="/content/tom/frame%d.jpg" % count;count+=1
        if not cv2.imwrite(filename, frame):
          raise Exception("Could not write image")
cap.release()
print ("Done!")

In [ ]:
# !ls -la tom

Οι εικόνες τώρα δημιουργήθηκαν. Ας δούμε μια εικόνα (σκηνή). Θα τη διαβάσουμε με την συνάρτηση `imread()` 

Let us try to visualize an image (frame). We will first read the image using the imread() function of matplotlib, and then plot it using the imshow() function.

In [ ]:
img = plt.imread('tom/frame0.jpg')   # reading image using its name
plt.imshow(img)

Αυτή είναι η πρώτη σκηνή από το βίντεο. Εξάγαμε μία σκηνή ανά δευτερόλεπτο, για όλη τη διάρκεια του βίντεο. Αφού η διάρκεια είναι 4:58 λεπτά (298 δευτερόλεπτα), έχουμε εξάγει συνολικά 298 εικόνες. 

Το πρόβλημά μας είναι να ταυτοποιήσουμε ποιές εικόνες έχουν τον Tom και ποιές τον Jerry. Αν οι εικόνες μας ήταν όμοιες με αυτές του Imagenet dataset, η διαδικασία διαχωρισμού θα ήταν εύκολη. Θα χρησιμοποιούσαμε κάποιο προ-εκπαιδευμένο μοντέλο στο Imagelnet και θα είχαμε και μεγάλο ποσοστό ακρίβειας! 

### Βήμα - 2: Χαρακτηρισμός κάποιων εικόνων για εκπαίδευση
Πως θα προχωρήσουμε; Μια λύση είναι να δώσουμε χειροκίνητα χαρακτηρισμούς σε μερικές από τις εικόνες μας, και με αυτές να εκπαιδεύσουμε ένα μοντέλο. Στη συνέχεια θα χρησιμοποιήσουμε αυτό το μοντέλο για να κάνουμε πρόβλεψη των υπόλοιπων εικόνων που δεν έχουν χρησιμοποιηθεί στην εκπαίδευση.

Ας σημειωθεί ότι μπορεί να υπάρχουν σκηνές που δεν υπάρχει κανείς από τους 2 χαρακτήρες. Έχουμε δηλαδή ένα πρόβλημα ταξινόμησης με πολλαπλές κλάσεις. Οι κλάσεις που έχουμε ορίσει εδώ είναι:

*  0 – ούτε JERRY ούτε TOM
*  1 – JERRY
*  2 – TOM

Ας διαβάσουμε το αρχείο mapping.csv με τους χαρακτηρισμούς:

In [ ]:
import pandas as pd
url_to_the_file  = "https://raw.githubusercontent.com/kmouts/PPS_MultiComms/master/mapping.csv"
data = pd.read_csv(url_to_the_file, sep=',')
data.head()      # printing first five rows of the file



Το αρχείο αντιστοίχισης έχει δύο στήλες: 

**Image_ID**: Το όνομα κάθε εικόνας
**Class**: Η κλάση
Το επόμενο βήμα είναι να διαβάσουμε τις εικόνες με βάση το όνομά τους (δηλ. η στήλη Image_ID column).

In [ ]:
X = [ ]     # creating an empty array
for img_name in data.Image_ID:
    img = plt.imread('tom/' + img_name)
    X.append(img)  # storing each image in array X
X = np.array(X)    # converting list to array

Έχουμε τώρα τις εικόνες σε μια λίστα. Θυμόμαστε ότι χρειαζόμαστε δύο πράγματα για να εκπαιδεύσουμε το μοντέλο μας:
*  Τις εικόνες εκπαίδευσης
*  Τις αντίστοιχες κλάσεις τους

Εφόσον έχουμε τρεις κλάσεις, θα τις κωδικοποιήσουμε (hot encode) με τη συνάρτηση `to_categorical()` από τη βιβλιοθήκη `keras.utils`.

In [ ]:
y = data.Class
dummy_y = np_utils.to_categorical(y)    # one hot encoding Classes

Θα χρησιμοποιήσουμε ένα προεκπαιδευμένο VGG16 μοντέλο, που παίρνει σαν είσοδο εικόνες μεγέθους (224 X 224 X 3). Εφόσον οι εικόνες μας είναι διαφορετικού μεγέθους, θα τους αλλάξουμε μέγεθος με τη συνάρτηση `resize()` από τη βιβλιοθήκη `skimage.transform`.

In [ ]:
image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224)).astype(int)      # reshaping to 224*224*3
    image.append(a)
X = np.array(image)

'Ολες οι εικόνες μας τώρα έχουμε μέγεθος 224 X 224 X 3. Αλλά πριν τις δώσουμε στο μοντέλο ως είσοδο, πρέπει να τις προεπεξεργαστούμε ανάλογα με τις απαιτήσεις του μοντέλου, αλλιώς δεν θα έχουμε καλή απόδοση. Εδώ χρησιμοποιούμε τη συνάρτηση  [`preprocess_input()`](https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/preprocess_input) από τη βιβλιοθήκη `keras.applications.vgg16`.

In [ ]:
from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X)      # preprocessing the input data

Χρειαζόμαστε και ένα σύνολο εικόνων επιβεβαίωσης (validation) για να ελέγχουμε την απόδοση του μοντέλου σε εικόνες που δεν έχει δει. Θα κάνουμε χρήση της συνάρτησης `train_test_split()` από το module `sklearn.model_selection` για τον τυχαίο διαχωρισμό των εικόνων σε τμήμα εκπαίδευσης και επιβεβαίωσης.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y, test_size=0.3, random_state=42)    # preparing the validation set

###Βήμα 3: Κατασκευάζοντας το μοντέλο
Θα κατασκευάσουμε το μοντέλο χρησιμοποιώντας, όπως είπαμε, ένα προεκπαιδευμένο  VGG16. Ας εισάγουμε τις απαραίτητες **βιβλιοθήκες**:

In [ ]:
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout

Εισάγουμε τώρα το προεκπαιδευμένο μοντέλο VGG16 και το αποθηκεύουμε ως βασικό μοντέλο:

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))    # include_top=False to remove the top layer

Θα κάνουμε προβλέψεις με χρήση αυτού του μοντέλου για `X_train` και `X_valid`, θα εξάγουμε τα χαρακτηριστικά, και στη συνέχεια θα χρησιμοποιήσουμε αυτά τα χαρακτηριστικά για να επανεκπαιδεύσουμε το μοντέλο.

In [ ]:
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
X_train.shape, X_valid.shape

Η μορφή των `X_train` και `X_valid` είναι (208, 7, 7, 512), (90, 7, 7, 512) αντίστοιχα. Για να τα περάσουμε στο νευρωνικό μας δίκτυο θα πρέπει να τα αναμορφώσουμε σε 1-D.

In [ ]:
X_train = X_train.reshape(208, 7*7*512)      # converting to 1-D
X_valid = X_valid.reshape(90, 7*7*512)

Τώρα θα προεπεξεργαστούμε τις εικόνες και θα τις κάνουμε να είναι επικεντρωμένες στο μηδέν (zero-centered) που βοηθά το μοντέλο να συγκλίνει γρηγορότερα.

In [ ]:
train = X_train/X_train.max()      # centering the data
X_valid = X_valid/X_train.max()

Τελικά, θα κατασκευάσουμε το μοντέλο μας. Αυτό χωρίζεται σε τρία βήματα:

1. Κατασκευή μοντέλου
2. Μεταγλώττιση (compiling) μοντέλου
3. Εκπαίδευση μοντέλου

In [ ]:
# i. Building the model
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='sigmoid')) # hidden layer
model.add(Dense(3, activation='softmax'))    # output layer

Ας δούμε τη σύνοψη του μοντέλου με χρήση της συνάρτησης `summary()`

In [ ]:
model.summary()

Έχουμε ένα κρυφό επίπεδο (hidden) με 1,024 νευρώνες και ένα επίπεδο εξόδου με 3 νευρώνες (αφού έχουμε 3 κλάσεις να προβλέψουμε). Ας μεταγλωττίσουμε το μοντέλο:

In [ ]:
# ii. Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Στο τελικό βήμα, θα εκπαιδεύσουμε το μοντέλο και παράλληλα θα ελέγουμε την απόδοσή του σε εικόνες που δεν έχει δει (το σύνολο εικόνων επιβεβαίωσης):

In [ ]:
# iii. Training the model
model.fit(train, y_train, epochs=50, validation_data=(X_valid, y_valid))

Μπορούμε να δούμε ότι η απόδοσή του είναι πολύ καλή στις εικόνες εκπαίδευσης και στις εικόνες επιβεβαίωσης. Έχουμε ακρίβεια πάνω από 86% σε εικόνες που δεν έχει δει. Και έτσι εκπαιδεύσαμε ένα μοντέλο σε δεδομένα από βίντεο για να παίρνουμε προβλέψεις για κάθε σκηνή.
Στο επόμενο βήμα, θα προσπαθήσουμε να υπολογίσουμε τον σκηνικό χρόνο των TOM και JERRY σε ένα νέο βίντεο.

 

##3. Υπολογισμός σκηνικού χρόνου – Μια απλή λύση
Θα κατεβάσουμε το νέο βίντεο όπως και πρίν, και θα εξάγουμε τις σκηνές:


In [ ]:
url = 'https://github.com/kmouts/PPS_MultiComms/blob/master/Tom_Jerry_3.mp4?raw=true'
output = 'Tom_jerry_3.mp4'
gdown.download(url, output, quiet=False)

In [ ]:
import os
if not os.path.isdir( 'tom3' ) :
    os.mkdir( 'tom3' )  # make sure the directory exists

In [ ]:
count = 0
videoFile = "Tom_jerry_3.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="tom3/test%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

In [ ]:
# ls -lah tom3

Μετά την εξαγωγή των σκηνών από το νέο βίντεο, θα φορτώσουμε το αρχείο `test.csv` που περιέχει τα ονόματα των εικόνων:

In [ ]:
url_to_the_file2  = "https://raw.githubusercontent.com/kmouts/PPS_MultiComms/master/test.csv"
test = pd.read_csv(url_to_the_file2, sep=',')
test.head()      # printing first five rows of the file



Στη συνέχεια, εισάγωουμε τις εικόνες προς δοκιμή και τις αλλάζουμε το μέγεθος σύμφωνα με τις απαιτήσεις του προεκπαιδευμένου μοντέλου:

In [ ]:
test_image = []
for img_name in test.Image_ID:
    img = plt.imread('tom3/' + img_name)
    test_image.append(img)
test_img = np.array(test_image)

In [ ]:
test_image = []
for i in range(0,test_img.shape[0]):
    a = resize(test_img[i], preserve_range=True, output_shape=(224,224)).astype(int)
    test_image.append(a)
test_image = np.array(test_image)

Θα χρειαστεί να κάνουμε και σε αυτές τις εικόνες τις αλλαγές που κάναμε και στις εικόνες εκπαίδευσης: θα τις προεπεξεργαστούμε, θα χρησιμοποιήσουμε τη συνάρτηση `base_model.predict()` για την εξαγωγή χαρακτηριστικών με το προεκπαιδευμένο μοντέλο VGG16, αλλαγή σε μορφή 1-D και zero-centered:

In [ ]:
# preprocessing the images
test_image = preprocess_input(test_image)

# extracting features from the images using pretrained model
test_image = base_model.predict(test_image)

# converting the images to 1-D form
test_image = test_image.reshape(186, 7*7*512)

# zero centered images
test_image = test_image/test_image.max()

Είμαστε έτοιμοι να χρησιμοποιήσουμε το μοντέλο που εκπαιδεύσαμε προηγουμένως για την πρόβλεψη αυτών των εικόνων.

### Βήμα – 4: Προβλέψεις για τις νέες εικόνες

In [ ]:
# predictions = model.predict_classes(test_image) # deprecated
predictions = np.argmax(model.predict(test_image), axis=-1)

### Βήμα – 5 Υπολογισμός σκηνικού χρόνου για TOM και JERRY
Θυμόμαστε πως η κλάση ‘1’ αντιπροσωπεύει παρουσία του JERRY, και ‘2’ του TOM. Θα χρησιμοποιήσουμε αυτές τις προγνώσεις για να υπολογίσουμε τον σκηνικό χρόνο τους:

In [ ]:
print("The screen time of JERRY is", predictions[predictions==1].shape[0], "seconds")
print("The screen time of TOM is", predictions[predictions==2].shape[0], "seconds")

##3. Σκέψεις για βελτίωση...
Πως θα μπορούσαμε να έχουμε καλύτερη απόδοση;

*   Καλύτερο μοντέλο
*   Βάρη στις κλάσεις
*   Checkpoint best model
*   Περισσότερες εικόνες εκπαίδευσης
*   Multi-class, multi-label problem: object detection
*   ...

